## Import Library and Dataset:

In [1]:
import os
import torch
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
from torch.optim import lr_scheduler

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import copy


In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
data_transforms = {
    'seg_train/seg_train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'seg_test/seg_test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = './intel/'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['seg_train/seg_train', 'seg_test/seg_test']}

train_sizes = len(image_datasets['seg_train/seg_train'])
test_sizes = len(image_datasets['seg_test/seg_test']) 

class_names = image_datasets['seg_train/seg_train'].classes
print(class_names)

['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']


In [4]:
art_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

art_dir = './'
art_datasets = {x: datasets.ImageFolder(os.path.join(art_dir, x), art_transforms[x]) for x in ['train', 'valid', 'test']}

train_sizes = len(art_datasets['train']) 
valid_sizes = len(art_datasets['valid']) 
test_sizes = len(art_datasets['test']) 

art_names = art_datasets['train'].classes
print(art_names)

['building', 'forest', 'mountain']


In [13]:
# load data                  
art_train_dataset = DataLoader(art_datasets['train'], batch_size=32,shuffle=True)
art_valid_dataset = DataLoader(art_datasets['valid'], batch_size=32,shuffle=True)
art_test_dataset = DataLoader(art_datasets['test'], batch_size=32,shuffle=True)

## Using pre-trained model from imageNet:

### Model: VGG19

In [28]:
model_vgg19 = models.vgg19(weights='DEFAULT').to(device)
for param in model_vgg19.features.parameters():
    param.requires_grad = False

In [29]:
model_vgg19.classifier[6] = nn.Linear(model_vgg19.classifier[6].in_features, len(art_names)).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_vgg19.classifier.parameters(), lr=0.0005)

In [22]:
model_vgg19

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

#### Freezed Model:

In [30]:
epochs = 20
best_acc = 0.0

for epoch in range(epochs):
    # train data
    model_vgg19.train()  
    for X, y in art_train_dataset:
        if torch.cuda.is_available():
            X, y = X.cuda(), y.cuda()
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward
        outputs = model_vgg19(X)
        loss = criterion(outputs, y)
        # backward + optimize 
        loss.backward()
        optimizer.step()

    # valid data
    valid_loss = 0.0
    correct = 0
    model_vgg19.eval()
    # Iterate over data.
    for X, y in art_valid_dataset:
        if torch.cuda.is_available():
            X, y = X.cuda(), y.cuda()
        # forward
        outputs = model_vgg19(X)
        loss = criterion(outputs, y)
        valid_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        correct += torch.sum(preds == y) 
    
    epoch_loss = valid_loss / len(art_valid_dataset)
    epoch_acc = correct / (len(art_valid_dataset) * 32)
    print(f'Epoch {epoch + 1}/{epochs}')
    print(f' Loss: {epoch_loss:.4f} Acc: {epoch_acc:.3f}')
    if epoch_acc >= best_acc:
        best_acc = epoch_acc
        torch.save(model_vgg19.state_dict(), 'freezed_vgg19_model.pt')

Epoch 1/20
 Loss: 0.9492 Acc: 0.562
Epoch 2/20
 Loss: 1.7787 Acc: 0.500
Epoch 3/20
 Loss: 0.9167 Acc: 0.688
Epoch 4/20
 Loss: 0.9054 Acc: 0.656
Epoch 5/20
 Loss: 0.6703 Acc: 0.750
Epoch 6/20
 Loss: 1.4295 Acc: 0.594
Epoch 7/20
 Loss: 1.1835 Acc: 0.656
Epoch 8/20
 Loss: 0.3525 Acc: 0.812
Epoch 9/20
 Loss: 1.2239 Acc: 0.719
Epoch 10/20
 Loss: 0.4936 Acc: 0.719
Epoch 11/20
 Loss: 1.2838 Acc: 0.688
Epoch 12/20
 Loss: 0.6276 Acc: 0.719
Epoch 13/20
 Loss: 0.5349 Acc: 0.781
Epoch 14/20
 Loss: 0.7001 Acc: 0.656
Epoch 15/20
 Loss: 0.4483 Acc: 0.719
Epoch 16/20
 Loss: 0.4555 Acc: 0.750
Epoch 17/20
 Loss: 0.9441 Acc: 0.625
Epoch 18/20
 Loss: 0.4753 Acc: 0.781
Epoch 19/20
 Loss: 1.0696 Acc: 0.688
Epoch 20/20
 Loss: 1.1867 Acc: 0.656


#### Unfreezed Model:

In [34]:
model_vgg19.load_state_dict(torch.load('freezed_vgg19_model.pt'))
model_vgg19 = model_vgg19.to(device)
for param in model_vgg19.parameters():
    param.requires_grad = True
unfreeze_criterion = nn.CrossEntropyLoss()
unfreeze_optimizer = torch.optim.SGD(model_vgg19.parameters(), lr=0.001, momentum=0.9)

In [38]:
epochs = 20
best_acc = 0.0

for epoch in range(epochs):
    # train data
    model_vgg19.train()  
    for X, y in art_train_dataset:
        if torch.cuda.is_available():
            X, y = X.cuda(), y.cuda()
        # zero the parameter gradients
        unfreeze_optimizer.zero_grad()
        # forward
        outputs = model_vgg19(X)
        loss = unfreeze_criterion(outputs, y)
        # backward + optimize 
        loss.backward()
        unfreeze_optimizer.step()

    # valid data
    valid_loss = 0.0
    correct = 0
    model_vgg19.eval()
    # Iterate over data.
    for X, y in art_valid_dataset:
        if torch.cuda.is_available():
            X, y = X.cuda(), y.cuda()
        # forward
        outputs = model_vgg19(X)
        loss = unfreeze_criterion(outputs, y)
        valid_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        correct += torch.sum(preds == y) 
    
    epoch_loss = valid_loss / len(art_valid_dataset)
    epoch_acc = correct / (len(art_valid_dataset) * 32)
    print(f'Epoch {epoch + 1}/{epochs}')
    print(f' Loss: {epoch_loss:.4f} Acc: {epoch_acc:.3f}')
    # stop training according to valid dataset
    if epoch_acc >= best_acc:
        best_acc = epoch_acc
        torch.save(model_vgg19.state_dict(), 'unfreeze_vgg19_model.pt')

Epoch 1/20
 Loss: 0.5709 Acc: 0.719
Epoch 2/20
 Loss: 0.8344 Acc: 0.812
Epoch 3/20
 Loss: 0.9485 Acc: 0.562
Epoch 4/20
 Loss: 0.6840 Acc: 0.719
Epoch 5/20
 Loss: 0.5871 Acc: 0.719
Epoch 6/20
 Loss: 0.7628 Acc: 0.719
Epoch 7/20
 Loss: 0.5368 Acc: 0.781
Epoch 8/20
 Loss: 0.3610 Acc: 0.812
Epoch 9/20
 Loss: 0.3807 Acc: 0.812
Epoch 10/20
 Loss: 0.5681 Acc: 0.781
Epoch 11/20
 Loss: 0.5133 Acc: 0.844
Epoch 12/20
 Loss: 0.5542 Acc: 0.812
Epoch 13/20
 Loss: 0.8281 Acc: 0.719
Epoch 14/20
 Loss: 0.6281 Acc: 0.781
Epoch 15/20
 Loss: 0.6981 Acc: 0.781
Epoch 16/20
 Loss: 0.7552 Acc: 0.781
Epoch 17/20
 Loss: 0.7273 Acc: 0.781
Epoch 18/20
 Loss: 0.7940 Acc: 0.781
Epoch 19/20
 Loss: 0.7566 Acc: 0.750
Epoch 20/20
 Loss: 0.5276 Acc: 0.750


### Pre-trained on intel image dataset: